# 1. Install AG2 + Materials Project dependencies

In [46]:
!pip install -U "ag2[openai]" autogen -q

import sys
!{sys.executable} -m pip install pymatgen mp-api numpy cython ipywidgets jupyterlab_widgets -q

print("✓ AG2/autogen + Pymatgen + MP-API installed.")

✓ AG2/autogen + Pymatgen + MP-API installed.


# 2. Imports + LLM Configuration


In [8]:
import os
import json
import random
from datetime import datetime, timedelta
from typing import Annotated, Any, Literal, Union, List, Optional, Tuple


# AG2 core imports
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import ReplyResult
from autogen.agentchat.group import (
    ContextVariables,
    AgentTarget, AgentNameTarget, StayTarget,
    OnCondition, StringLLMCondition,
    OnContextCondition, ExpressionContextCondition, ContextExpression,
    RevertToUserTarget, TerminateTarget
)
from autogen.agentchat.groupchat import GroupChat, GroupChatManager
from autogen.tools import tool


# Tools + External APIs
from mp_api.client import MPRester


# Coding tools
from autogen.coding.local_commandline_code_executor import LocalCommandLineCodeExecutor
from autogen.coding.base import CodeBlock
from pydantic import BaseModel, Field

from autogen.agentchat.group.patterns import DefaultPattern
from autogen.agentchat import initiate_group_chat


# LLM configuration

llm_config = LLMConfig(config_list={
    "api_type": "openai",
    "model": "gpt-4o",
    "api_key": os.environ["OPENAI_API_KEY"],
})

print("✓ Imports loaded and LLM configured.")


✓ Imports loaded and LLM configured.


In [10]:
import autogen
print(autogen.__version__)


0.10.0


# 3. User query + Context Variables

In [13]:
from autogen.agentchat.group import ContextVariables

# Development mode switch
DEV_MODE = False   # False for real user input

# Initialize context variables
context_variables = ContextVariables(
    data={
        "user_query": None,
        "explained_terms": None,
        "mp_query": None,
        "mp_results": None,
        "final_conclusion": None,
    }
)

print("✓ Context variables initialized.")

✓ Context variables initialized.


# 4. TOOLS (A → B → C → D)

In [16]:
# Tool A — explain_query

def explain_query_tool(query: Annotated[str, "initial query AS provided by the user without modification."], 
                       query_explanation: Annotated[str, "An explaination of the query, including a breakdown of key and secondary terms, 2-3 sentences."],
                       context_variables: ContextVariables)-> ReplyResult:
    '''Explain the posed query. What does it ask? Include a breakdown of the key and secondary terms in the query.'''

    if not isinstance(query, str):
        raise ValueError("The 'query' parameter must be a string.")

    if len(query.strip()) == 0:
        raise ValueError("Query cannot be empty.")

    explanation = (
        f"Explanation of user query: '{query}'\n\n"
        f"- Query explanation: {query_explanation}"
    )

    context_variables["task_started"] = True
    context_variables["user_query"] = query
    context_variables["explained_terms"] = query_explanation

    target_agent = AgentNameTarget('AgentB_MaterialsRetriever') #check names match
        
    return ReplyResult(
        message=explanation,
        target=target_agent,
        context_variables=context_variables,
    )


# Tool B — download_materials_structures_properties_from_mp

def material_retiever(search_criteria: Annotated[dict, "Filtering conditions used to select candidate materials."],
                                                     fields: Annotated[List[str], "List of metadata fields to retrieve for each material."],
                                                     sample_number: Annotated[int, "Number of materials to randomly sample and download from the filtered set."],
                                                     context_variables: ContextVariables) -> ReplyResult:
    "Retrieve materials from Materials Project using search_criteria, fields, and sample_number. Then, produce a structured analysis of the retrieved materials."""

    # --- 1) Query Materials Project ---

    search_kwargs = dict(search_criteria) if search_criteria is not None else {}

    api_key = os.getenv("MP_API_KEY")
    with MPRester(api_key) as mpr:
        all_results = list(
            mpr.materials.summary.search(
                fields=fields,
                **search_kwargs,
            )
        )

    # Simple empty-case handling
    if not all_results:
        message = "No materials found for the given search_criteria. Consider relaxing your filters."
        return ReplyResult(
            message=message,
            target=AgentNameTarget("Human"),
            context_variables=context_variables,
        )

    # Sampling
    if sample_number is not None and sample_number > 0 and sample_number < len(all_results):
        results = random.sample(all_results, sample_number)
    else:
        results = all_results

    # Save raw query + results into context
    context_variables["mp_query"] = {
        "search_criteria": search_criteria,
        "fields": fields,
        "sample_number": sample_number,
    }
    context_variables["mp_results"] = results

    # --- 2) Build structured analysis ---

    structured_output = []

    for entry in results:
        # entry is usually a SummaryDoc-like object
        def get(k):
            if isinstance(entry, dict):
                return entry.get(k)
            return getattr(entry, k, None)

        material_block = {
            "Material": {
                "ID": get("material_id"),
                "Formula": get("formula_pretty") or get("pretty_formula"),
                "Bandgap": get("band_gap"),
                "Density": get("density"),
                "Volume": get("volume"),
                "Energy_above_hull": get("energy_above_hull") or get("e_above_hull"),
            },
            "Usage": {
                "Applications": "Potential applications based on retrieved properties.",
                "Industry_relevance": "Why this material could matter for industry.",
            },
            "Sustainability": {
                "Abundance": "Placeholder for abundance / criticality analysis.",
                "Environmental_impact": "Placeholder evaluation of toxicity / extraction cost.",
                "Recyclability": "Placeholder for recyclability commentary.",
            },
            "Weaknesses": {
                "Structural_limitations": "Placeholder for mechanical or thermal limitations.",
                "Economic_limitations": "Placeholder for cost or manufacturability issues.",
            },
        }

        structured_output.append(material_block)

    # Save structured analysis into context for other agents
    context_variables["final_conclusion"] = structured_output

    # --- 3) Build final *string* message for ReplyResult ---

    import json

    header = f"Retrieved {len(results)} materials from Materials Project and produced a structured analysis.\n"
    analysis_text = json.dumps(structured_output, indent=2, default=str)

    message = header + "\nStructured analysis (JSON):\n" + analysis_text

    target_agent = AgentNameTarget("Human")  # check names match

    return ReplyResult(
        message=message,   # now a string ✅
        target=target_agent,
        context_variables=context_variables,
    )


# Tool C — python_coder

project_folder = os.path.abspath("ag2_project")
os.makedirs(project_folder, exist_ok=True)


class PythonCode(BaseModel):
    code: str = Field(..., description="Full python code executed by the coder agent")


def python_coder_tool(code: Annotated[str, "A single block of Python code to execute."],
                      file_name: Annotated[str, "Name of the code file to store the executed script, e.g., 'script.py'"],
                      context_variables: ContextVariables) -> ReplyResult:
    """ Execute Python code, save it to ag2_project/<file_name>, update a persistent JSON context, and return the execution output."""

    target = AgentNameTarget("AgentD_Coder")

    # Build full code with shebang
    full_code = "#!/usr/bin/env python3\n" + code
    python_code_model = PythonCode(code=full_code)

    #Execute code
    executor = LocalCommandLineCodeExecutor(timeout=60000)
    code_block = CodeBlock(language="python", code=full_code)
    result = executor.execute_code_blocks([code_block])

    exit_code = result.exit_code
    output = result.output

    # Load or initialize persistent context file
    context_path = os.path.join(project_folder, "context_variables_data.json")

    try:
        with open(context_path, "r") as f:
            context_file_data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        context_file_data = {
            "execution_results": {},
            "execution_history": [],
            "execution_notes": [],
        }

    # Update context based on exit_code 
    if exit_code != 0:
        context_file_data["code_error"] = python_code_model.model_dump()
    else:
        if "code" not in context_file_data or not isinstance(
            context_file_data.get("code", []), list
        ):
            context_file_data["code"] = []
        context_file_data["code"].append(python_code_model.model_dump())
        context_file_data["code_error"] = ""

    # Save executed code 
    code_output_path = os.path.join(project_folder, file_name)
    os.makedirs(os.path.dirname(code_output_path), exist_ok=True)

    with open(code_output_path, "w") as f:
        f.write(full_code)

    # Remove temporary file created by executor (if any) 
    try:
        if result.code_file and os.path.exists(result.code_file):
            os.remove(result.code_file)
    except Exception:
        pass

    # Save persistent JSON context file 
    with open(context_path, "w") as f:
        json.dump(context_file_data, f, indent=2)

    # Also store the last run in the in-memory context_variables for other agents
    context_variables["last_executed_code"] = full_code
    context_variables["last_execution_output"] = output

    # Build return message 
    if exit_code == 0:
        message = (
            "Code executed successfully.\n"
            f"Output:\n{output}\n\n"
            "Proceed with another python_coder_tool call if needed."
        )
    else:
        message = (
            "Code execution failed.\n"
            f"Error output:\n{output}\n\n"
            "Correct the code and call python_coder_tool again."
        )

    target_agent = AgentNameTarget('Human') #check names match

    return ReplyResult(
        message=message,
        target=target_agent,
        context_variables=context_variables,
    )

# 5. Manual tests for Materials Project tool


In [19]:
print("Running manual tests for material_retiever...\n")

from autogen.agentchat.group import ContextVariables

# Test 1 — Simple valid query
try:
    search_criteria_1 = {
        "band_gap": [3, 5],
        "energy_above_hull": (0, 0.1),
    }
    fields_1 = ["material_id", "formula_pretty", "band_gap", "energy_above_hull"]
    sample_number_1 = 5

    ctx_1 = ContextVariables(data={})
    result_1 = material_retiever(
        search_criteria=search_criteria_1,
        fields=fields_1,
        sample_number=sample_number_1,
        context_variables=ctx_1,
    )
    print("Test 1 completed. Message:")
    print(result_1.message)
except Exception as e:
    print("Test 1 failed with unexpected exception:", e)


# Test 2 — Empty search_criteria (edge case, but allowed for now)
try:
    search_criteria_2 = {}
    fields_2 = ["material_id"]
    sample_number_2 = 3

    ctx_2 = ContextVariables(data={})
    result_2 = material_retiever(
        search_criteria=search_criteria_2,
        fields=fields_2,
        sample_number=sample_number_2,
        context_variables=ctx_2,
    )
    print("\nTest 2 completed (empty search_criteria). Message:")
    print(result_2.message)
except Exception as e:
    print("Test 2 raised exception (edge case, no validation yet):", e)


# Test 3 — Empty fields list (likely to cause MP API error)
try:
    search_criteria_3 = {
        "band_gap": (1, 3),
    }
    fields_3 = []
    sample_number_3 = 3

    ctx_3 = ContextVariables(data={})
    result_3 = material_retiever(
        search_criteria=search_criteria_3,
        fields=fields_3,
        sample_number=sample_number_3,
        context_variables=ctx_3,
    )
    print("\nTest 3 completed (empty fields). Message:")
    print(result_3.message)
except Exception as e:
    print("Test 3 raised exception (expected until we add validation):", e)


# Test 4 — sample_number <= 0 (edge case)
try:
    search_criteria_4 = {
        "band_gap": (0, 10),
    }
    fields_4 = ["material_id"]
    sample_number_4 = 0

    ctx_4 = ContextVariables(data={})
    result_4 = material_retiever(
        search_criteria=search_criteria_4,
        fields=fields_4,
        sample_number=sample_number_4,
        context_variables=ctx_4,
    )
    print("\nTest 4 completed (sample_number <= 0). Message:")
    print(result_4.message)
except Exception as e:
    print("Test 4 raised exception (edge case, no validation yet):", e)


# Test 5 — Invalid filter key (no validation yet, just see what MP does)
try:
    search_criteria_5 = {
        "invalid_property": (0, 1),
    }
    fields_5 = ["material_id"]
    sample_number_5 = 3

    ctx_5 = ContextVariables(data={})
    result_5 = material_retiever(
        search_criteria=search_criteria_5,
        fields=fields_5,
        sample_number=sample_number_5,
        context_variables=ctx_5,
    )
    print("\nTest 5 completed (invalid filter key). Message:")
    print(result_5.message)
except Exception as e:
    print("Test 5 raised exception (this will guide future validation):", e)


print("\nManual tool tests completed.")

Running manual tests for material_retiever...



Retrieving SummaryDoc documents:   0%|          | 0/14851 [00:00<?, ?it/s]

Test 1 completed. Message:
Retrieved 5 materials from Materials Project and produced a structured analysis.

Structured analysis (JSON):
[
  {
    "Material": {
      "ID": "mp-1234155",
      "Formula": "Sm2MgMo2(I4O15)2",
      "Bandgap": 3.1158,
      "Density": null,
      "Volume": null,
      "Energy_above_hull": 0.043233700174405004
    },
    "Usage": {
      "Applications": "Potential applications based on retrieved properties.",
      "Industry_relevance": "Why this material could matter for industry."
    },
    "Sustainability": {
      "Abundance": "Placeholder for abundance / criticality analysis.",
      "Environmental_impact": "Placeholder evaluation of toxicity / extraction cost.",
      "Recyclability": "Placeholder for recyclability commentary."
    },
    "Weaknesses": {
      "Structural_limitations": "Placeholder for mechanical or thermal limitations.",
      "Economic_limitations": "Placeholder for cost or manufacturability issues."
    }
  },
  {
    "Material":

/opt/anaconda3/envs/wildfiresai/lib/python3.11/site-packages/mp_api/client/core/client.py:515: UserWarning: Ignoring `fields` argument: All fields are always included when no query is provided.
  warnings.warn(
/opt/anaconda3/envs/wildfiresai/lib/python3.11/site-packages/mp_api/client/core/client.py:1326: UserWarning: Omitting a query also includes deprecated documents in the results. Make sure to post-filter them out.
  warnings.warn(


Retrieving SummaryDoc documents:   0%|          | 0/164969 [00:00<?, ?it/s]

...

Retrieving SummaryDoc documents:   0%|          | 0/34231 [00:00<?, ?it/s]

/opt/anaconda3/envs/wildfiresai/lib/python3.11/site-packages/mp_api/client/core/client.py:825: UserWarning: Use the 'fields' argument to select only fields of interest to speed up data retrieval for large queries. Choose from: ['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'property_name', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetizati


Test 3 completed (empty fields). Message:
Retrieved 3 materials from Materials Project and produced a structured analysis.

Structured analysis (JSON):
[
  {
    "Material": {
      "ID": "mp-1202225",
      "Formula": "MgSO6",
      "Bandgap": 1.7555,
      "Density": 1.9090544291343132,
      "Volume": 1060.2541479215265,
      "Energy_above_hull": 0.252173064843749
    },
    "Usage": {
      "Applications": "Potential applications based on retrieved properties.",
      "Industry_relevance": "Why this material could matter for industry."
    },
    "Sustainability": {
      "Abundance": "Placeholder for abundance / criticality analysis.",
      "Environmental_impact": "Placeholder evaluation of toxicity / extraction cost.",
      "Recyclability": "Placeholder for recyclability commentary."
    },
    "Weaknesses": {
      "Structural_limitations": "Placeholder for mechanical or thermal limitations.",
      "Economic_limitations": "Placeholder for cost or manufacturability issues."


Retrieving SummaryDoc documents:   0%|          | 0/154873 [00:00<?, ?it/s]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



# 6. AGENTS (A → B → C → D + HUMAN)

In [22]:
#Agent A: Explainer

explainer_message = """
You are an explainer AI agent.

### ROLE
You interpret the user's materials-science question and produce a structured scientific explanation.

### YOUR TASKS
- Parse the user's query from context_variables["user_query"].
- Identify relevant scientific terminology (e.g., bandgap, symmetry, density, conductivity).
- Generate a clear explanation of what the user is asking.
- Clarify key terms in simple but technically correct language.

### TOOL USAGE
- You must call the tool: explain_query_tool to provide your explanation.
"""

AgentA_Explainer = ConversableAgent(
    name="AgentA_Explainer",
    llm_config=llm_config,
    system_message=explainer_message,
    human_input_mode="NEVER",
    functions=[explain_query_tool],
)


#Agent B: Materials Retriever 

retriever_message = """
You are a material retriever AI agent.

### ROLE
You transform the explained query into a valid Materials Project query and then summarize the retrieved materials.

### YOUR TASKS
- Read the explanation produced by AgentA_Explainer.
- Construct a valid Materials Project API search request using search_criteria + fields + sample_number.
- Call the tool to:
  - retrieve candidate materials from Materials Project, and
  - produce a structured analysis of the retrieved materials.
- Store results in context_variables["mp_results"] and context_variables["final_conclusion"].

### TOOL USAGE
- You must call the tool: material_retiever to find and analyze materials.
- Be careful about the keys in search_criteria, e.g.:
  - Allowed numeric filters: band_gap, energy_above_hull, k_voigt, g_voigt, num_sites
  - Allowed composition filters: elements, chemsys, excluded_elements
"""

AgentB_MaterialsRetriever = ConversableAgent(
    name="AgentB_MaterialsRetriever",
    llm_config=llm_config,
    system_message=retriever_message,
    human_input_mode="NEVER",
    functions=[material_retiever],
)



#Agent C: Analyzer

analyzer_message = """
You are an analyser AI agent.

### ROLE
You interpret and refine the analysis of the retrieved Materials Project results.

### YOUR TASKS
- Read the raw MP results from context_variables["mp_results"].
- Read the structured analysis from context_variables["final_conclusion"].
- Optionally reorganize, clarify, or deepen the scientific discussion of those results.
- Produce a clear, human-readable summary and recommendations for the Human.

### OUTPUT
- A concise but technically accurate explanation of the materials,
  their properties, potential applications, and limitations.
"""

AgentC_Analyzer = ConversableAgent(
    name="AgentC_Analyzer",
    llm_config=llm_config,
    system_message=analyzer_message,
    human_input_mode="NEVER",
    # No tools for now; you can add one later if needed
)


#Agent D: Coding Agent

coder_message = """
You are a coder AI agent.

### ROLE
Generate and execute Python code when requested by the other agents.

### YOUR TASKS
- Write Python functions, calculations, or plots.
- Execute Python code strictly through the python_coder_tool.
- Save executed code into ag2_project/<file_name>.
- Return only clean, concise execution results (via the tool output).
"""

AgentD_Coder = ConversableAgent(
    name="AgentD_Coder",
    llm_config=llm_config,
    system_message=coder_message,
    human_input_mode="NEVER",
    functions=[python_coder_tool],
)


#Human Agent:

Human = ConversableAgent(
    name="Human",
    human_input_mode="ALWAYS",
)

# 7. Pattern definition 


In [25]:
# Set up the conversation pattern

pattern = DefaultPattern(
    initial_agent=AgentA_Explainer,
    user_agent=Human,
    group_after_work=Human,
    agents=[AgentA_Explainer,
            AgentB_MaterialsRetriever,
            AgentC_Analyzer,
            AgentD_Coder,
        ],
    context_variables=context_variables,

)

# 8. Group Chat

In [28]:
result, context, _ = initiate_group_chat(
pattern=pattern,
messages='Give a material with band_gap greater than 3',
max_rounds=20,
)

Human (to chat_manager):

Give a material with band_gap greater than 3

--------------------------------------------------------------------------------

Next speaker: AgentA_Explainer

AgentA_Explainer (to chat_manager):

***** Suggested tool call (call_fxp9adEUE6PsjLmevAtIwhlJ): explain_query_tool *****
Arguments: 
{"query":"Give a material with band_gap greater than 3","query_explanation":"The user is asking for a material that has a bandgap greater than 3 electron volts (eV). The term 'bandgap' refers to the energy difference between the top of the valence band and the bottom of the conduction band in a material. This property determines a material's electrical conductivity and its ability to act as an insulator or a semiconductor. A bandgap above 3 eV typically indicates that the material behaves as an insulator."}
***********************************************************************************

--------------------------------------------------------------------------------

N

AttributeError: 'ConversableAgent' object has no attribute 'resolve'